In [1]:
from semantic_kernel.agents import AgentGroupChat
from semantic_kernel.agents.strategies import TerminationStrategy

In [2]:
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.functions import kernel_function
from semantic_kernel.connectors.ai import FunctionChoiceBehavior

from typing import Annotated

In [3]:
import json
import os
from dotenv import load_dotenv
import psycopg2
from psycopg2.extras import RealDictCursor
load_dotenv()

True

In [4]:
def _create_kernel_with_chat_completion(service_id: str) -> Kernel:
    kernel = Kernel()
    kernel.add_service(AzureChatCompletion(service_id=service_id, 
                                        api_key=os.getenv("AZURE_OPENAI_API_KEY"),
                                        deployment_name=os.getenv("AZURE_OPENAI_CHAT_COMPLETION_MODEL"),
                                        endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                                        ))
    return kernel

## Termination Strategies

**KernelFunctionTerminationStrategy**: This strategy uses a kernel function to determine when to terminate the conversation. It allows developers to define custom termination logic using a prompt.

**MaxIterationsTerminationStrategy**: Terminates the conversation after a specified number of iterations to prevent infinite loops.

**GoalCompletionTerminationStrategy**: This strategy terminates tasks once predefined outcomes are achieved, such as successfully retrieving requested data.

**ApprovalTerminationStrategy**: Termination based on Approval of the orchestrator agent.

**KeywordTerminationStrategy**: Termination based on certain keyword in the discussion.

In [5]:
class ApprovalTerminationStrategy(TerminationStrategy):
    """A strategy for determining when an agent should terminate."""

    async def should_agent_terminate(self, agent, history):
        """Check if the agent should terminate."""
        return "approved" in history[-1].content.lower()

In [6]:
from decimal import Decimal

class DatabaseConnector:
    def __init__(self):
        self.create_connection()

    @kernel_function(description="Create a connection object to the postgres database.")
    def create_connection(self):
        print("create_connection function called... ")
        connection = psycopg2.connect(
            dbname=os.getenv("DB_NAME"),
            user=os.getenv("DB_USER"),
            password=os.getenv("DB_PASSWORD"),
            host=os.getenv("DB_HOST"),
            port=os.getenv("DB_PORT")
        )
        self.connection = connection
        self.cursor = self.connection.cursor(cursor_factory=RealDictCursor)

    @kernel_function(description="Fetches data based on the query provided.")
    def query_database(self, query: Annotated[str, "query to be executed"]) -> Annotated[str, "Returns the queried information as a json"]:
        """
        Fetches the information from the required table in PostgreSQL database.

        :param query (str): the query to be executed.
        :return: fetched information as a JSON string.
        :rtype: str
        """
        print("query_database function called... query: ", query)
        try:
            # if(self.connection != True):
            #     self.create_connection()
            
            #connection = self.connection
            cursor = self.cursor
            cursor.execute(query=query)
            result_record = cursor.fetchone()
            if result_record:
                # Convert Decimal values to strings
                for key, value in result_record.items():
                    if isinstance(value, Decimal):
                        result_record[key] = str(value)
                return json.dumps({"result_record": result_record})
            else:
                return json.dumps({"error": "An error occured while fetching the data."})
        except Exception as e:
            return json.dumps({"error": str(e)})
        # finally:
        #     if connection:
        #         self.close_connection()
        
    
    @kernel_function(description="Closes the connection to the database.")
    def close_connection(self) -> Annotated[str, "Returns a message indicating the status of the connection closure."]:
        """
        Closes the connection to the PostgreSQL database.

        :return: Message indicating the status of the connection closure.
        :rtype: str
        """
        print("close_connection function called... ")
        try:
            if self.connection:
                self.cursor.close()
                self.connection.close()
            return "Connection closed successfully."
        except Exception as e:
            return str(e)

In [7]:
acc = DatabaseConnector()
acc.create_connection()
print(acc.query_database("select * from accounts where account_id = '1'"))
acc.close_connection()

create_connection function called... 
create_connection function called... 
query_database function called... query:  select * from accounts where account_id = '1'
{"result_record": {"account_id": 1, "name": "HBL", "email": "abcd@gef.com", "phone": "9876543210", "balance": "99034.50"}}
close_connection function called... 


'Connection closed successfully.'

In [8]:
def _create_kernel_with_chat_completion_and_plugin(service_id: str) -> Kernel:
    kernel = Kernel()
    kernel.add_service(AzureChatCompletion(service_id=service_id, 
                                        api_key=os.getenv("AZURE_OPENAI_API_KEY"),
                                        deployment_name=os.getenv("AZURE_OPENAI_CHAT_COMPLETION_MODEL"),
                                        endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
                                        ))

    kernel.add_plugin(DatabaseConnector(), plugin_name="db_connector")
    return kernel

In [9]:
TRANSCRIPTION_REVIEWER = "TranscriptionReviewer"
TRANSCRIPTION_REVIEWER_INSTRUCTIONS = """
You are a transcription reviewer who has been tasked with reviewing a conversation between a customer service agent and a customer.
The goal is to examine the conversation and determine if the customer's card is blocked. 
If so, extract the customer id from the transcript and provide it to the Orchestrator.
if not, provide a response stating that you are skiping this transcript.
"""

ANALYST_NAME = "BusinessAnalyst"
ANALYST_INSTRUCTIONS = """
You are a highly skilled business analyst with extensive experience in writing and executing SQL queries. Fix errors in the SQL queries and ensure they are correct.
Do not make any assumptions about query results. You need to execute necessary SQL queries to determine the reason for a blocked card.
You have access to a PostgreSQL database that contains information about customers and their credit card transactions.
The database contains information about customers in the table customerdata and their credit card transactions in the table credit_card_transactions.
The table customerdata has the following columns: customer_id: , card_blocked, payment_due, card_type and credit_card_no.
The table credit_card_transactions has the following columns: credit_card_no, date, amount, authentication_passed and location.

You have been provided with a customer id by the Orchestrator and you need to query the database to determine the reason for the blocked card.
You need to check the card_blocked column in the customerdata table to determine if the card is blocked. 
If so, you need to check the credit_card_transactions table to determine if the card is blocked due to more than 3 authentication failures. 
If there were more than 3 authentication failures recently, the card is blocked because of that. 
If not, the card is blocked due to some other reason. 
You need to provide the reason (authentication failure or unknown reason) to the Orchestrator.
"""

ORCHESTRATOR_NAME = "Orchestrator"
ORCHESTRATOR_INSTRUCTIONS = """
You are an orchestrator who has been tasked with coordinating the work of the transcription reviewer and the business analyst.
You will receive the customer id from the transcription reviewer and pass it to the business analyst and ask it to determine the reason for the blocked card by executing the necessary SQL queries.
If the reviwer provides a response stating that the card is not blocked, you will conclude that the transcript is being skipped as it is not related to Card Blocking.
After the analysis is provided by Business Analyst, you will draft an email to the customer with the analysis provided by the business analyst.
After you have drafted the email, you MUST approve the analysis by using keywords such as "approved" or "not approved". 
"""

TASK = "Read transcript and determine the reason for blocked card"

In [10]:
# 1. Create the reviewer agent based on the chat completion service
agent_reviewer = ChatCompletionAgent(
    kernel=_create_kernel_with_chat_completion("reviewer"),
    name=TRANSCRIPTION_REVIEWER,
    instructions=TRANSCRIPTION_REVIEWER_INSTRUCTIONS,
)

In [11]:
# 2. Create the copywriter agent based on the chat completion service
analyst_kernel = _create_kernel_with_chat_completion_and_plugin("business_analyst")
settings = analyst_kernel.get_prompt_execution_settings_from_service_id(service_id="business_analyst")
settings.function_choice_behavior = FunctionChoiceBehavior.Auto()
agent_analyst = ChatCompletionAgent(
    kernel=analyst_kernel,
    name=ANALYST_NAME,
    instructions=ANALYST_INSTRUCTIONS,
    
)

create_connection function called... 


In [12]:
# 3. Create the reviewer agent based on the chat completion service
agent_orchestrator = ChatCompletionAgent(
    kernel=_create_kernel_with_chat_completion("orchestrator"),
    name=ORCHESTRATOR_NAME,
    instructions=ORCHESTRATOR_INSTRUCTIONS,
)

In [13]:
# 4. Place the agents in a group chat with a custom termination strategy
group_chat = AgentGroupChat(
    agents=[
        agent_reviewer,
        agent_analyst,
        agent_orchestrator 
        
    ],
    termination_strategy=ApprovalTerminationStrategy(
        agents=[agent_orchestrator],
        maximum_iterations=25,
    ),
)

In [14]:
from semantic_kernel.agents.group_chat.agent_chat import ChatMessageContent

message_content = ChatMessageContent(role="user", content=""" 
                                     Customer ID : 123456
Date: 16-03-2025 10:23:30

Call Transcript:
            
Agent: how can I help you today? 
Customer: My card is blocked, could you please help me with that?
Agent: I am sorry to hear that. Can you please provide me with your name, contact number, email id and address to unblock the card?
Customer: My name is Vikas, contact number is 1234567890, email id is vsdsdf@gmail.com and address is 1234, 5th Avenue, New York, NY 10001
Agent: Thank you for providing the details. I have raised a request for this to be looked at immediately. You will receive a confirmation email shortly. Is there anything else I can help you with?
Customer: No, that's all. Thank you.
Agent: You are welcome. Have a great day!
                                     
                                     """)

In [15]:
# 4. Add the task as a message to the group chat
await group_chat.add_chat_message(message_content)


In [16]:
# 5. Invoke the chat
async for content in group_chat.invoke():
    print(f"# {content.name}: {content.content}")


# TranscriptionReviewer: The customer's card is indeed blocked. Extracting the customer ID:

**Customer ID: 123456**

Submitting the customer ID to the Orchestrator.
query_database function called... query:  SELECT card_blocked FROM customerdata WHERE customer_id = 123456;
query_database function called... query:  SELECT COUNT(*) as auth_failures FROM credit_card_transactions WHERE credit_card_no IN (SELECT credit_card_no FROM customerdata WHERE customer_id = 123456) AND authentication_passed = false;
query_database function called... query:  SELECT COUNT(*) as auth_failures FROM credit_card_transactions WHERE credit_card_no IN (SELECT CAST(credit_card_no AS TEXT) FROM customerdata WHERE customer_id = 123456) AND authentication_passed = false;
query_database function called... query:  ROLLBACK;
query_database function called... query:  SELECT card_blocked FROM customerdata WHERE customer_id = 123456;
query_database function called... query:  SELECT COUNT(*) as auth_failures FROM credit